In [ ]:
import pandas as pd

import http.client
import json

In [ ]:
# Configuration de la connexion
conn = http.client.HTTPSConnection("google.serper.dev")

# Clé API et CID de l'établissement
API_KEY = ""

CID_dict = {
    "16105863628885510708": "UGC Ciné Cité Les Halles",
    "14031669981001205": "UGC Ciné Cité La Défense",
    "4594451141627986672": "UGC Ciné Cité Bercy",
    "5717521880024175258": "UGC Montparnasse",
    "18376887391546998649": "UGC Ciné Cité Paris 19",
    "11973972513515530793": "UGC Lyon Bastille",
    "18070383662248289434": "UGC Rotonde",
    "2434503191443723831": "UGC Danton",
    "15974422746109813532": "UGC Gobelin",
    "4875917883076010033": "UGC Opera",
    "4670800676058725171": "UGC Issy-les-Moulineaux",
    "10714513477244355524": "UGC cité Rosny",
    "2531614383931627138": "UGC O'Parinor",
    "3591339807304781443": "UGC Créteil",
    "18100841303958206733": "UGC Noisy-le-Grand",
    "9397722383352281642": "UGC cité Cergy-le-Haut",
    "14266124050547786911": "UGC Enghien",
    "9884127632012507889": "UGC cité Vélizy",
    "9450560118436865956": "UGC Odéon",
    "1297730194394241617": "UGC cité Maillot",
    "12066373839463197372": "UGC Roxane",
    "12561263782760200041": "UGC Cyrano",
    "12066066470647964123": "UGC les Ulis",
    "11528741487733739147": "UGC Plaisir",
    "4827495686679570626": "UGC Ciné Cité SQY Ouest",
    "1545734532206739981": "UGC Le Majestic",
    "7758509024386655426": "UGC Talence",
    "1063231029971688648": "UGC Ciné Cité Bassins à Flot",
    "1221074619586416348": "UGC Ciné Cité Bordeaux Gambetta",
    "6150073597153944739": "UGC Ciné Cité Mondeville",
    "1297730194394241617": "UGC Ciné Cité Internationale",
    "11007085165420456664": "UGC Ciné Cité Confluence",
    "16480571364716580805": "UGC Astoria",
    "5237829782691708778": "UGC Ciné Cité Part-Dieu",
    "1580738400597029145": "UGC Ciné Cité Lille",
    "13851014890891807582": "UGC Ciné Cité Villeneuve d'Ascq",
    "182429166120925991": "UGC Nancy Saint Jean",
    "571204129825596177": "UGC Ciné Cité Ludres",
    "235404832692447283": "UGC Ciné Cité Atlantis",
    "14886586888478736724": "UGC Ciné Cité Strasbourg",
    "16970239325385179790": "UGC Montaudran"
}

# Variables pour stocker les avis et gérer la pagination
reviews_data = []
next_page_token = None

In [ ]:
for CID, CID_name in CID_dict.items():    
    for language in ["fr", "en"]:
        while True:
            # Préparation du payload pour chaque requête de page
            payload = {
                "cid": CID,
                "hl": language,
                "sortBy": "newest"
            }
            if next_page_token:
                payload["nextPageToken"] = next_page_token

            headers = {
                'X-API-KEY': API_KEY,
                'Content-Type': 'application/json'
            }

            # Envoi de la requête
            conn.request("POST", "/reviews", json.dumps(payload), headers)
            res = conn.getresponse()
            data = res.read()

            # Décodage et conversion en JSON
            data_json = json.loads(data.decode("utf-8"))

            # Vérification si des avis sont présents
            if "reviews" in data_json:
                for review in data_json["reviews"]:
                    # Récupération des informations de chaque avis et utilisateur
                    reviews_data.append({
                        'rating': review.get('rating'),
                        'date': review.get('isoDate'),
                        'review': review.get('snippet'),
                        'likes': review.get('likes'),
                        'user_name': review.get('user', {}).get('name'),
                        'user_nb_reviews': review.get('user', {}).get('reviews'),
                        'user_nb_photos': review.get('user', {}).get('photos'),
                        'cinéma': CID_name,
                        'language': language
                    })

                # Mise à jour du jeton pour la page suivante
                next_page_token = data_json.get("nextPageToken")

                # Si pas de page suivante, sortir de la boucle
                if not next_page_token:
                    break
            else:
                print("Erreur lors de la récupération des avis ou fin des avis.")
                break